Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


# linear_programming_example

First, you must install [ortools](https://pypi.org/project/ortools/) package in this colab.

In [ ]:
%pip install ortools


Linear optimization example.

In [4]:
from ortools.sat.python import cp_model

# Datos del problema
num_trabajadores = 10
trabajadores = range(num_trabajadores)

# Tipos de turno y requerimientos por trabajador
tipos_turno = ['wm', 'wa', 'wn', 'wkm', 'wkn', 'hm', 'hn']

# Turnos requeridos por cada trabajador para cada tipo
req_por_trabajador = {
    'wm': 32,
    'wa': 50,
    'wn': 48,
    'wkm': 20,
    'wkn': 24,
    'hm': 2,
    'hn': 2
}

# Número de trabajadores que se deben asignar en cada turno
trabajadores_por_turno = {
    'wm': 1,
    'wa': 2,
    'wn': 2,
    'wkm': 1,
    'wkn': 2,
    'hm': 1,
    'hn': 2
}

# Determinamos el número total de turnos de cada tipo en el calendario.
turnos_totales = {t: (req_por_trabajador[t] * num_trabajadores) // trabajadores_por_turno[t]
                  for t in tipos_turno}

# Creación del modelo
model = cp_model.CpModel()

# Variable de asignación
asignacion = {}
for t in tipos_turno:
    asignacion[t] = {}
    for i in range(turnos_totales[t]):
        asignacion[t][i] = {}
        for w in trabajadores:
            asignacion[t][i][w] = model.NewBoolVar(f'asign_{t}_{i}_{w}')

# Restricción 1: Cada turno debe cubrirse con el número exacto de trabajadores necesarios.
for t in tipos_turno:
    for i in range(turnos_totales[t]):
        model.Add(sum(asignacion[t][i][w] for w in trabajadores) == trabajadores_por_turno[t])

# Restricción 2: Cada trabajador debe tener asignado el número exacto de turnos de cada tipo.
for w in trabajadores:
    for t in tipos_turno:
        model.Add(sum(asignacion[t][i][w] for i in range(turnos_totales[t])) == req_por_trabajador[t])

# Restricción 3: Incentivar la diversidad en las parejas de trabajo
parejas = {}
for i in trabajadores:
    for j in trabajadores:
        if i < j:
            parejas[(i, j)] = []
            for t in tipos_turno:
                if trabajadores_por_turno[t] > 1:
                    for k in range(turnos_totales[t]):
                        pareja_var = model.NewBoolVar(f'pareja_{t}_{k}_{i}_{j}')
                        model.Add(asignacion[t][k][i] + asignacion[t][k][j] == 2).OnlyEnforceIf(pareja_var)
                        model.Add(asignacion[t][k][i] + asignacion[t][k][j] != 2).OnlyEnforceIf(pareja_var.Not())
                        parejas[(i, j)].append(pareja_var)

# Corregir la función objetivo: aplanar la lista antes de sumar
todas_parejas = [var for plist in parejas.values() for var in plist]
model.Maximize(sum(todas_parejas))

# Resolver el modelo
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    for t in tipos_turno:
        print(f'\nTurno {t}:')
        for i in range(turnos_totales[t]):
            asignados = [w for w in trabajadores if solver.Value(asignacion[t][i][w]) == 1]
            print(f'  Turno {i}: Trabajadores asignados: {asignados}')
else:
    print("No se encontró solución factible.")



Turno wm:
  Turno 0: Trabajadores asignados: [9]
  Turno 1: Trabajadores asignados: [6]
  Turno 2: Trabajadores asignados: [8]
  Turno 3: Trabajadores asignados: [6]
  Turno 4: Trabajadores asignados: [9]
  Turno 5: Trabajadores asignados: [7]
  Turno 6: Trabajadores asignados: [7]
  Turno 7: Trabajadores asignados: [9]
  Turno 8: Trabajadores asignados: [7]
  Turno 9: Trabajadores asignados: [8]
  Turno 10: Trabajadores asignados: [5]
  Turno 11: Trabajadores asignados: [1]
  Turno 12: Trabajadores asignados: [8]
  Turno 13: Trabajadores asignados: [3]
  Turno 14: Trabajadores asignados: [6]
  Turno 15: Trabajadores asignados: [6]
  Turno 16: Trabajadores asignados: [9]
  Turno 17: Trabajadores asignados: [8]
  Turno 18: Trabajadores asignados: [5]
  Turno 19: Trabajadores asignados: [2]
  Turno 20: Trabajadores asignados: [4]
  Turno 21: Trabajadores asignados: [3]
  Turno 22: Trabajadores asignados: [2]
  Turno 23: Trabajadores asignados: [8]
  Turno 24: Trabajadores asignados: [0]